# RNN
循环神经网络

## 阅读材料

[Fundamentals of Deep Learning – Introduction to Recurrent Neural Networks](https://www.analyticsvidhya.com/blog/2017/12/introduction-to-recurrent-neural-networks/)


## 笔记

1. A single time step of the input is supplied to the network i.e. xt is supplied to the network
2. We then calculate its current state using a combination of the current input and the previous state i.e. we calculate ht
3. The current ht becomes ht-1 for the next time step
4. We can go as many time steps as the problem demands and combine the information from all the previous states
5. Once all the time steps are completed the final current state is used to calculate the output yt
6. The output is then compared to the actual output and the error is generated
7. The error is then backpropagated to the network to update the weights(we shall go into the details of backpropagation in further sections) and the network is trained

## 简单实现

样本用'hello'这个单词，目标是输入'hell',能推理出'o'

In [1]:
word = 'hello'
letters = list(word)
print(f"letters:{letters}")
words = list(set(letters))
voc_size = len(words)
print(f"vocabular:{words}")
print(f"voc size:{voc_size}")

letters:['h', 'e', 'l', 'l', 'o']
vocabular:['h', 'e', 'o', 'l']
voc size:4


In [2]:
# 保持和文章一致方便对照，这里硬编码word to idx
word_to_idx = {
    'h':0,
    'e':1,
    'l':2,
    'o':3
}
print(word_to_idx)

{'h': 1, 'e': 2, 'l': 3, 'o': 4}


In [3]:
import torch

def one_hot_encoding(word):
    tensor = torch.zeros(len(word_to_idx))
    tensor[word_to_idx[word]] = 1
    return tensor

encoded_words = [(one_hot_encoding(word),word)for word in words]
print(encoded_words)

IndexError: index 4 is out of bounds for dimension 0 with size 4

In [ ]:
Wxh = torch.tensor([
    [0.287027,0.84606,0.572392,0.486813],
    [0.902874,0.871522,0.691079,0.18998],
    [0.537524,0.09224,0.558159,0.491528]
],dtype=torch.float32)

print(Wxh)

In [ ]:
Xt = torch.tensor([1,0,0,0],dtype=torch.float32)
print(Xt)
print(Xt.shape)
# print(Xt.view(-1,1))

result = torch.matmul(Wxh,Xt)

print(result)
print(result.shape)
result1 = torch.matmul(Wxh,Xt.view(-1,1))
print(result1)
print(result1.shape)


In [ ]:
# 探索一下矩阵和行向量，gpt生成了一个例子

# 定义一个1x3的矩阵
matrix = torch.tensor([[1, 2, 3]])

# 定义一个dim为3的行向量
row_vector = torch.tensor([1, 2, 3])

print(matrix)
print(row_vector)

> 在这个示例中，我们定义了一个1x3的矩阵matrix和一个dim为3的行向量row_vector，它们在大多数情况下可以等价使用。

> 然而，在一些情况下，例如矩阵乘法等操作中，PyTorch可能会对它们进行不同的处理，因此在进行特定操作时可能需要注意它们的类型和维度。但在大多数情况下，这两种表示都可以等价使用。


In [ ]:
# 随机初始化测试

rnd = torch.rand(1,1)

print(rnd)

In [ ]:
# 随机初始变量
Whh = torch.rand(1,1)
bias = torch.rand(1,1)

print(Whh)
print(bias)

In [ ]:
# 测试一下计算

# state = torch.matmul(Whh,result1).add(bias)

print(Whh * result1 + bias)
#print(state)

In [ ]:
print(torch.tanh(Whh * result1 + bias))

对原文的`Why`有疑惑，还有计算的完整过程也有没理解的地方，用文中数据实际进行计算来反推

进行完整的过程计算

In [ ]:
Whh = torch.tensor([[0.427043]])
bias = torch.tensor([[0.57700]])

print(f"Whh={Whh},bias={bias}")

In [ ]:
# 第一步计算时，前一个状态是没有的，所以用[0,0,0]

h0 = torch.tensor([[0],[0],[0]])
print(h0)

In [ ]:
def calc(previous_state):
    return Whh * previous_state + bias

In [ ]:
# step 1,计算 'h'
calced_h0 = calc(h0)
print(calced_h0)

Wxh_Xt = torch.matmul(Wxh,one_hot_encoding('h').view(-1,1))
print(Wxh_Xt)

print(Wxh)

# h1 = torch.tanh( calc(h0)+ torch.matmul(Wxh,one_hot_encoding('h').view(-1,1)))
# print(h1)


In [ ]:
# step 2，计算 'e'
h2 = torch.tanh( calc(h1)+ torch.matmul(Wxh,one_hot_encoding('e').view(-1,1)))
print(h2)

In [ ]:
# step 3，计算 'l'
h3 = torch.tanh( calc(h2)+ torch.matmul(Wxh,one_hot_encoding('l').view(-1,1)))
print(h3)

In [ ]:
# step 4，计算 'l'
h4 = torch.tanh( calc(h3)+ torch.matmul(Wxh,one_hot_encoding('l').view(-1,1)))
print(h4)